In [1]:
import sys
sys.path.append('C:\\Users\\dlfdus\\PycharmProjects\\mypacakge\\')
from corpus import DoublespaceLineCorpus
from corpus import DoublespaceLineCorpusWithList
from tqdm import tqdm_notebook
import requests
import pandas as pd
import re
import pickle
from collections import Counter
from scipy import spatial

In [2]:
import gensim
from scipy.io import mmwrite, mmread

tokenized_corpus_fname = 'data/tmp/doc2vec_word_data.txt' 

C:\Users\dlfdus\Anaconda3\lib\site-packages\gensim\utils.py:865: UserWarning: detected Windows; aliasing chunkize to chunkize_serial
  warnings.warn("detected Windows; aliasing chunkize to chunkize_serial")


In [3]:
corpus_ = open(tokenized_corpus_fname, 'r')
line = corpus_.readlines()

In [4]:
reviews = []
for idx, i in enumerate(line):
    temp = i.split('\t')[1]
    temp = re.sub('\n', '', temp)
    reviews.append(temp)

In [5]:
from gensim.models import Doc2Vec
from gensim.models.doc2vec import TaggedDocument

def TaggedReview(reviews):
    for doc in reviews:
        company_tag, review = doc.split('\t')
        text = re.sub('\n', '', review)
        yield TaggedDocument(words=text.split(), tags=['Company_%s' % company_tag])

doc2vec_corpus = list(TaggedReview(line))

In [6]:
for num_doc, doc in enumerate(doc2vec_corpus):
    if num_doc > 5: break
    print(doc)

TaggedDocument(['업계', '1위', '하나투어', '인정', '많음', '여행사', '친구들', '연봉', '복지', '좋음'], ['Company_(주)하나투어'])
TaggedDocument(['여행', '혜택', '연차', '사용', '있음', '가족', '균형', '측면', '좋음'], ['Company_(주)하나투어'])
TaggedDocument(['연차', '회사', '커뮤니케이션'], ['Company_(주)하나투어'])
TaggedDocument(['브랜드', '1위', '저녁', '여행', '할인'], ['Company_(주)하나투어'])
TaggedDocument(['있음', '업무', '찾기', '힘듬'], ['Company_(주)하나투어'])
TaggedDocument(['연차', '사용', '비교적', '회사', '복지', '거점', '운영', '시차', '근무', '재택근무', '직원', '제공', '상품', '직원', '이용', '가능', '분위기', '정시퇴근', '승진', '어려움', '없음'], ['Company_(주)하나투어'])


In [7]:
doc2vec_corpus[:5]

[TaggedDocument(words=['업계', '1위', '하나투어', '인정', '많음', '여행사', '친구들', '연봉', '복지', '좋음'], tags=['Company_(주)하나투어']),
 TaggedDocument(words=['여행', '혜택', '연차', '사용', '있음', '가족', '균형', '측면', '좋음'], tags=['Company_(주)하나투어']),
 TaggedDocument(words=['연차', '회사', '커뮤니케이션'], tags=['Company_(주)하나투어']),
 TaggedDocument(words=['브랜드', '1위', '저녁', '여행', '할인'], tags=['Company_(주)하나투어']),
 TaggedDocument(words=['있음', '업무', '찾기', '힘듬'], tags=['Company_(주)하나투어'])]

In [8]:
train_Doc2Vec = False

In [9]:
%%time
doc2vec_fname = 'data/tmp/doc2vec_model.pkl' 
if train_Doc2Vec:
    doc2vec_model = Doc2Vec(doc2vec_corpus, size=200, alpha=0.05, window=5, min_count=5, negative=5) 
    with open(doc2vec_fname, 'wb') as f:
        pickle.dump(doc2vec_model, f)

else:
    with open(doc2vec_fname, 'rb') as f:
        doc2vec_model = pickle.load(f)

Wall time: 67.8 ms


In [10]:
index2tag = [doc2vec_model.docvecs.index_to_doctag(i) for i in range(654)]
print(index2tag[:5])

tag2index = {tag:index for index, tag in enumerate(index2tag)}

['Company_(주)하나투어', 'Company_호텔신라(주)', 'Company_대명레저산업(주)', 'Company_이랜드파크(주)', 'Company_모두투어네트워크(주)']


In [11]:
index2name = []

for i in index2tag:
    index2name.append(i.split('_')[1])
print(index2name[:5])

['(주)하나투어', '호텔신라(주)', '대명레저산업(주)', '이랜드파크(주)', '모두투어네트워크(주)']


In [12]:
from scipy import spatial
tag_list = list(tag2index.keys())

In [13]:
from engine import tokenizer
from engine import sentiment_analysis

with open('data/tmp/lr.pkl', 'rb') as f:
    model = pickle.load(f)
with open('data/tmp/vectorizer.pkl', 'rb') as f:
    vectorizer = pickle.load(f)
with open('data/tmp/company_IF.pkl', 'rb') as f:
    company_IF = pickle.load(f)

In [14]:
P_doc2vec_fname = 'data/tmp/P_doc2vec_model.pkl' 
N_doc2vec_fname = 'data/tmp/N_doc2vec_model.pkl' 

with open(P_doc2vec_fname, 'rb') as f:
    P_doc2vec_model = pickle.load(f)
with open(N_doc2vec_fname, 'rb') as f:
    N_doc2vec_model = pickle.load(f)

In [15]:
tokenizer('업무보다는 정치질이 우선인 회사 발전 가능성이 없음')

'업무 보다는 정치질이 우선 회사 발전 가능 성이 없음'

In [16]:
tokenizer('업무보다는 정치질이 우선인 회사 발전 가능성이 없음', only_noun = True)

['업무', '우선', '회사', '발전', '가능', '없음']

In [32]:
sentiment_analysis(model, vectorizer)

장점인지는 모르겠는데 일이 생각보다 많음 적은인원으로 개처럼 일해야함
 


'부정 문장'

In [18]:
industry_field_1 = list(company_IF['1차산업군'].unique())
industry_field_2 = list(company_IF['2차산업군'].unique())

In [19]:
company_field_1 = {}
company_field_2 = {}

for i in tag_list:
    i = i.split('_')[1]
    company_field_1[i] = company_IF['1차산업군'][company_IF['회사명']==i].values[0]
    company_field_2[i] = company_IF['2차산업군'][company_IF['회사명']==i].values[0]

In [20]:
field1_field2 = {}

for i in industry_field_1:
    field1_field2[i] = list(company_IF['2차산업군'][company_IF['1차산업군']==i].unique())

In [21]:
def permutations_(po, ne):
    permutation = []
    for i in po:
        for j in ne:
            if i != j:
                permutation.append((i,j))
    return permutation

In [22]:
def calculate_distance(tag_list, polarity, topn=10):
    if polarity == 'P':
        comment = input('\n원하는 회사의 장점\n')
        tokenized_comment = tokenizer(comment, only_noun = True)
        inferred_vector = P_doc2vec_model.infer_vector(tokenized_comment)
        most_similar, most_similar_company = [], []
        count = 0
        while count < topn+1:
            similar_ = 1000
            company_name = ''
            for company_ in tag_list:
                distance = spatial.distance.cosine(inferred_vector, P_doc2vec_model.docvecs[company_])
                if distance < similar_ and company_ not in most_similar_company:
                    similar_ = distance
                    company_name = company_
            most_similar.append(similar_)
            most_similar_company.append(company_name)
            count += 1
            if count == topn:
                break
        return most_similar_company
    elif polarity == 'N':
        comment = input('\n피하고 싶은 회사의 단점\n')
        tokenized_comment = tokenizer(comment, only_noun = True)
        inferred_vector = N_doc2vec_model.infer_vector(tokenized_comment)
        most_similar, most_similar_company = [], []
        count = 0
        while count < topn+1:
            similar_ = 1000
            company_name = ''
            for company_ in tag_list:
                distance = spatial.distance.cosine(inferred_vector, N_doc2vec_model.docvecs[company_])
                if distance < similar_ and company_ not in most_similar_company:
                    similar_ = distance
                    company_name = company_
            most_similar.append(similar_)
            most_similar_company.append(company_name)
            count += 1
            if count == topn:
                break
        return most_similar_company

In [60]:
def find_my_company(topn=5, set_polarity = 'both'):
    tag_list = list(tag2index.keys())
    
    field1 = input('1차 산업군 선택 : \n{}\n'.format(industry_field_1))
    if field1 == '전체 선택':
        target_company = tag_list
        print('\n검색 대상 회사 : \n{}'.format('전체'))
        if set_polarity == 'both':
            p_ = calculate_distance(tag_list, 'P', topn=len(target_company))
            n_ = calculate_distance(tag_list, 'N', topn=len(target_company))
        else:
            p_ = calculate_distance(tag_list, 'P', topn=len(target_company))
            n_ = [0]
            
    elif field1 in industry_field_1:
        field2 = input('\n2차 산업군 선택 : \n{}\n'.format(field1_field2[field1]))
        if field2 == '전체 선택':
            target_company = list(company_IF['회사명'][(company_IF['1차산업군']==field1)].values)
            print('\n검색 대상 회사 : {} 개 회사 \n{}'.format(len(target_company),target_company))
            tag_list = ['Company_'+i for i in target_company]
            if set_polarity == 'both':
                p_ = calculate_distance(tag_list, 'P', topn=len(target_company))
                n_ = calculate_distance(tag_list, 'N', topn=len(target_company))
            else:
                p_ = calculate_distance(tag_list, 'P', topn=len(target_company))
                n_ = [0]
            #return p_, n_
        
        elif field2 in industry_field_2:
            target_company = list(company_IF['회사명'][(company_IF['1차산업군']==field1)&(company_IF['2차산업군']==field2)].values)
            print('\n검색 대상 회사 : {} 개 회사 \n{}'.format(len(target_company),target_company))
            tag_list = ['Company_'+i for i in target_company]
            if set_polarity == 'both':
                p_ = calculate_distance(tag_list, 'P', topn=len(target_company))
                n_ = calculate_distance(tag_list, 'N', topn=len(target_company))
            else:
                p_ = calculate_distance(tag_list, 'P', topn=len(target_company))
                n_ = [0]
        else:
            print('\n옳바른 2차 산업군을 입력해주세요')  
    else:
        print('\n옳바른 1차 산업군을 입력해주세요')
        
    recom_distance_list = []
    company_list = []
    recommend = {}
    
    
    for permutation in permutations_(p_, n_):
        
        if set_polarity == 'both': most_similar = doc2vec_model.docvecs.most_similar(positive=[permutation[0]], negative=[permutation[1]], topn=624)
        else: most_similar = doc2vec_model.docvecs.most_similar(positive=[permutation[0]], topn=624)
        
        for company_ in most_similar:
            if company_[0] in tag_list and company_[0] not in company_list:
                recom_distance_list.append(company_[1])
                company_list.append(company_[0])
            
    for idx, i in enumerate(company_list):
        recommend[i] = recom_distance_list[idx]
    recommend = sorted(recommend.items(), key=lambda x:x[1], reverse=True)[:topn]
    #return recommend
    
    for idx, i in enumerate(recommend):
        print('\n', idx+1, i[0])

In [61]:
similar_company_list= find_my_company(5, set_polarity = 'positive')

1차 산업군 선택 : 
['서비스업', '제조/화학', '의료/제약/복지', '유통/무역/운송', '교육업', '건설업', 'IT/웹/통신', '미디어/디자인', '은행/금융업', '기관/협회']
제조/화학

2차 산업군 선택 : 
['전기/전자/제어', '석유/화학/에너지', '식품가공/개발', '반도체/광학/디스플레이', '섬유/의류/패션', '환경', '기계/설비/자동차', '가구/목재/제지', '조선/항공/우주', '화장품/뷰티', '금속/재료/철강/요업', '생활용품/소비재/사무', '기타']
전기/전자/제어

검색 대상 회사 : 28 개 회사 
['삼성전자(주)', '엘지이노텍(주)', '삼성에스디아이(주)', '팬택(주)', '삼성전기(주)', '한국단자공업(주)', '코리아써키트(주)', '히로세코리아(주)', '(주)케이엠더블유', '네패스(주)', '(주)인터플렉스', '(주)심텍홀딩스', '아모텍(주)', '져스텍(주)', '(주)인바디', '(주)오토닉스', '대덕지디에스(주)', '영풍전자(주)', '홍콩상하이은행', '(주)에이비비코리아', '타이코에이엠피(주)', '대한전선(주)', '유비쿼스(주)', '한화시스템(주)', '파트론(주)', '한국동서발전(주)', '크루셜텍(주)', '일진전기(주)']

원하는 회사의 장점
인간적인 사무실 분위기, 뛰어난 복지, 자부심이 느껴지는 곳

 1 Company_팬택(주)

 2 Company_(주)케이엠더블유

 3 Company_크루셜텍(주)

 4 Company_대한전선(주)

 5 Company_유비쿼스(주)


In [53]:
similar_company_list

[('Company_크루셜텍(주)', 0.7547199726104736),
 ('Company_팬택(주)', 0.7547199726104736),
 ('Company_(주)오토닉스', 0.6644994020462036),
 ('Company_유비쿼스(주)', 0.6606053113937378),
 ('Company_대한전선(주)', 0.6569345593452454)]

In [43]:
similar_company_list= find_my_company(10, set_polarity = 'both')

1차 산업군 선택 : 
['서비스업', '제조/화학', '의료/제약/복지', '유통/무역/운송', '교육업', '건설업', 'IT/웹/통신', '미디어/디자인', '은행/금융업', '기관/협회']
제조/화학

2차 산업군 선택 : 
['전기/전자/제어', '석유/화학/에너지', '식품가공/개발', '반도체/광학/디스플레이', '섬유/의류/패션', '환경', '기계/설비/자동차', '가구/목재/제지', '조선/항공/우주', '화장품/뷰티', '금속/재료/철강/요업', '생활용품/소비재/사무', '기타']
전기/전자/제어

검색 대상 회사 : 28 개 회사 
['삼성전자(주)', '엘지이노텍(주)', '삼성에스디아이(주)', '팬택(주)', '삼성전기(주)', '한국단자공업(주)', '코리아써키트(주)', '히로세코리아(주)', '(주)케이엠더블유', '네패스(주)', '(주)인터플렉스', '(주)심텍홀딩스', '아모텍(주)', '져스텍(주)', '(주)인바디', '(주)오토닉스', '대덕지디에스(주)', '영풍전자(주)', '홍콩상하이은행', '(주)에이비비코리아', '타이코에이엠피(주)', '대한전선(주)', '유비쿼스(주)', '한화시스템(주)', '파트론(주)', '한국동서발전(주)', '크루셜텍(주)', '일진전기(주)']

원하는 회사의 장점
뛰어난 동료와 사무실 분위기

피하고 싶은 회사의 단점
적은 연봉


In [44]:
similar_company_list

[('Company_크루셜텍(주)', 0.28467464447021484),
 ('Company_팬택(주)', 0.23650842905044556),
 ('Company_(주)인바디', 0.17774799466133118),
 ('Company_삼성전자(주)', 0.1325964778661728),
 ('Company_유비쿼스(주)', 0.120412677526474),
 ('Company_대한전선(주)', 0.1093357726931572),
 ('Company_홍콩상하이은행', 0.1017989069223404),
 ('Company_코리아써키트(주)', 0.09539346396923065),
 ('Company_(주)오토닉스', 0.03739279881119728),
 ('Company_(주)인터플렉스', 0.03360418975353241)]

In [62]:
similar_company_list= find_my_company(5, set_polarity = 'both')

1차 산업군 선택 : 
['서비스업', '제조/화학', '의료/제약/복지', '유통/무역/운송', '교육업', '건설업', 'IT/웹/통신', '미디어/디자인', '은행/금융업', '기관/협회']
서비스업

2차 산업군 선택 : 
['호텔/여행/항공', '외식업/식음료', 'AS/카센터/주유', '뷰티/미용', '시설관리/용역/헤드헌팅', '레저/스포츠/여가', '기타']
호텔/여행/항공

검색 대상 회사 : 11 개 회사 
['(주)하나투어', '호텔신라(주)', '대명레저산업(주)', '이랜드파크(주)', '모두투어네트워크(주)', '(주)노랑풍선', '한화호텔앤드리조트(주)', '내일투어(주)', '레드캡투어(주)', '(유)에스아이에프씨호텔디벨로프먼트', '제주항공(주)']

원하는 회사의 장점
인간적인 분위기 뛰어난 복지 일과 삶의 균형

피하고 싶은 회사의 단점
고객의 갑질 과도한 업무

 1 Company_제주항공(주)

 2 Company_레드캡투어(주)

 3 Company_내일투어(주)

 4 Company_(주)노랑풍선

 5 Company_모두투어네트워크(주)


In [26]:
similar_company_list= find_my_company(5, set_polarity = 'both')

1차 산업군 선택 : 
['서비스업', '제조/화학', '의료/제약/복지', '유통/무역/운송', '교육업', '건설업', 'IT/웹/통신', '미디어/디자인', '은행/금융업', '기관/협회']
서비스업

2차 산업군 선택 : 
['호텔/여행/항공', '외식업/식음료', 'AS/카센터/주유', '뷰티/미용', '시설관리/용역/헤드헌팅', '레저/스포츠/여가', '기타']
호텔/여행/항공

검색 대상 회사 : 11 개 회사 
['(주)하나투어', '호텔신라(주)', '대명레저산업(주)', '이랜드파크(주)', '모두투어네트워크(주)', '(주)노랑풍선', '한화호텔앤드리조트(주)', '내일투어(주)', '레드캡투어(주)', '(유)에스아이에프씨호텔디벨로프먼트', '제주항공(주)']

원하는 회사의 장점
인간적인 분위기 뛰어난 복지 일과 삶의 균형

피하고 싶은 회사의 단점
고객의 갑질 과도한 업무


In [27]:
similar_company_list

[('Company_호텔신라(주)', 0.14878416061401367),
 ('Company_한화호텔앤드리조트(주)', 0.1375071108341217),
 ('Company_이랜드파크(주)', 0.08093768358230591),
 ('Company_제주항공(주)', -0.0027748597785830498),
 ('Company_(유)에스아이에프씨호텔디벨로프먼트', -0.005606256425380707)]